In [1]:
import re
import math
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer
from collections import defaultdict
import os
from scipy.sparse import csr_matrix

# File paths
document_folder = "./data"  # Folder containing text documents
stopword_file = './stopwords.txt'  # Stopword list

# Load stopwords
with open(stopword_file, 'r') as f:
    stop_words = set(line.strip().lower() for line in f if line.strip())

def tokenization(text):
    # 使用正則表達式一次性移除標點和數字
    doc = re.sub(r"[^\w\s]|[\d]", " ", text)  # 移除標點符號和數字
    
    # 將文本轉為小寫
    doc = doc.lower()
    
    # Tokenization: 使用 split() 切分並清理多餘空白
    words = [word.strip() for word in doc.split() if word.strip()]
    
    # Porter's stemming
    stemmer = PorterStemmer()
    stemming = [stemmer.stem(word) for word in words]
    
    # Stop words removal
    token_list = [word for word in stemming if word not in stop_words]
    
    return token_list

# Load and preprocess documents
documents = []
for i in range(1, 1096):  # Document IDs range from 1 to 1095
    file_path = os.path.join(document_folder, f"{i}.txt")
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        processed_content = tokenization(content)
        documents.append(processed_content)


In [2]:

# Compute document frequency (DF) for terms
term_df = defaultdict(int)
for doc in documents:
    unique_terms = set(doc)
    for term in unique_terms:
        term_df[term] += 1

# Sort terms alphabetically and assign indices
sorted_terms = sorted(term_df.items(), key=lambda x: x[0])
dictionary_entries = [(idx + 1, term, df) for idx, (term, df) in enumerate(sorted_terms)]

# Compute IDF values
N = 1095  # Total number of documents
idf_dict = {}
for idx, term, df in dictionary_entries:
    idf_value = math.log10(N / df) if df > 0 else 0
    idf_dict[term] = (idx, idf_value)

# Compute normalized TF-IDF vectors
sparse_tfidf_vectors = []
for doc in documents:
    term_count = defaultdict(int)
    for term in doc:
        term_count[term] += 1

    total_terms = len(doc)
    tfidf_vector = {}
    for term, count in term_count.items():
        tf = count / total_terms
        idx, idf_value = idf_dict.get(term, (0, 0))
        tfidf_vector[idx] = tf * idf_value

    # Normalize TF-IDF vector
    magnitude = np.linalg.norm(list(tfidf_vector.values()))
    if magnitude > 0:
        tfidf_vector = {idx: value / magnitude for idx, value in tfidf_vector.items()}

    sparse_tfidf_vectors.append(tfidf_vector)

# Convert sparse vectors to a dense matrix
all_indices = sorted(idf_dict.values(), key=lambda x: x[0])
tfidf_matrix = np.zeros((len(documents), len(all_indices)))
for doc_idx, tfidf_vector in enumerate(sparse_tfidf_vectors):
    for term_idx, value in tfidf_vector.items():

        tfidf_matrix[doc_idx, term_idx - 1] = value
        
min_term_idx = float('inf')  # 設置一個很大的初始值
for doc_idx, tfidf_vector in enumerate(sparse_tfidf_vectors):
    for term_idx, value in tfidf_vector.items():
        if term_idx < min_term_idx:
            min_term_idx = term_idx

print(f"The smallest term index is: {min_term_idx}")


The smallest term index is: 1


In [3]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    def _parent(self, index):
        return (index - 1) // 2

    def _left_child(self, index):
        return 2 * index + 1

    def _right_child(self, index):
        return 2 * index + 2

    def _heapify_up(self, index):
        while index > 0 and self.heap[self._parent(index)][0] < self.heap[index][0]:
            self.heap[self._parent(index)], self.heap[index] = self.heap[index], self.heap[self._parent(index)]
            index = self._parent(index)

    def _heapify_down(self, index):
        largest = index
        left = self._left_child(index)
        right = self._right_child(index)

        if left < len(self.heap) and self.heap[left][0] > self.heap[largest][0]:
            largest = left
        if right < len(self.heap) and self.heap[right][0] > self.heap[largest][0]:
            largest = right
        if largest != index:
            self.heap[index], self.heap[largest] = self.heap[largest], self.heap[index]
            self._heapify_down(largest)

    def push(self, similarity, cluster1, cluster2):
        self.heap.append((similarity, cluster1, cluster2))
        self._heapify_up(len(self.heap) - 1)

    def pop(self):
        if len(self.heap) == 0:
            return None
        if len(self.heap) == 1:
            return self.heap.pop()
        max_value = self.heap[0]
        self.heap[0] = self.heap.pop()
        self._heapify_down(0)
        return max_value

    def peek(self):
        return self.heap[0] if self.heap else None

    def size(self):
        return len(self.heap)

    def is_empty(self):
        return len(self.heap) == 0



In [4]:
def compute_cosine_similarity(matrix):
    N = matrix.shape[0]
    similarity_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i == j:
                similarity_matrix[i, j] = 1.0  # Cosine similarity with itself
            else:
                dot_product = np.dot(matrix[i], matrix[j])
                norm_i = np.linalg.norm(matrix[i])
                norm_j = np.linalg.norm(matrix[j])
                if norm_i > 0 and norm_j > 0:
                    similarity_matrix[i, j] = dot_product / (norm_i * norm_j)
                else:
                    similarity_matrix[i, j] = 0.0
    return similarity_matrix


similarity_matrix = compute_cosine_similarity(tfidf_matrix)
print(similarity_matrix[0, 1])

0.2032423017240074


In [8]:


def hierarchical_clustering(tfidf_matrix, similarity_matrix, K, method='complete-link'):
    N = tfidf_matrix.shape[0]
    print(N)

    # Initialize clusters
    clusters = {i: [i] for i in range(N)}

    # Initialize custom heap
    max_heap = MaxHeap()
    for i in range(N):
        for j in range(i + 1, N):
            similarity = similarity_matrix[i, j]
            max_heap.push(similarity, i, j)

    while len(clusters) > K:
        # Extract the most similar clusters
        s, cluster1, cluster2 = max_heap.pop()
        #print(s)

        # Ensure clusters are still valid
        if cluster1 in clusters and cluster2 in clusters:
            # Merge clusters
            clusters[cluster1].extend(clusters[cluster2])
            del clusters[cluster2]
            #print(f"Current content of cluster1 (after merge): {clusters[cluster1]}")

            # Recalculate similarity with other clusters
            for other_cluster in list(clusters.keys()):
                if other_cluster != cluster1:
                    if method == 'single-link':
                        sim = max(
                            similarity_matrix[i, j]
                            for i in clusters[cluster1]
                            for j in clusters[other_cluster]
                        )
                    elif method == 'complete-link':
                        sim = min(
                            similarity_matrix[i, j]
                            for i in clusters[cluster1]
                            for j in clusters[other_cluster]
                        )
                    max_heap.push(sim, cluster1, other_cluster)

    return clusters

In [19]:
def save_clusters(clusters, file_name):
    # Sort the clusters by the minimum document ID in each cluster
    sorted_clusters = {k: sorted(v) for k, v in clusters.items()}
    sorted_clusters = dict(sorted(sorted_clusters.items(), key=lambda x: min(x[1])))

    with open(file_name, 'w') as f:
        for cluster_id, doc_ids in sorted_clusters.items():
            # Write doc_ids in a straight line, one per line
            for doc_id in doc_ids:
                f.write(f"{doc_id+1}\n")
            # Add an empty line after each cluster
            f.write("\n")

# Perform clustering for K=8, 13, 20
for K in [8, 13, 20]:
    clusters = hierarchical_clustering(tfidf_matrix, similarity_matrix, K)
    save_clusters(clusters, f"{K}.txt")



1095
1095
1095
